In [1]:
import numpy as np
from astropy.io import fits


### Retrieving the relevant CCD Data

In [2]:
decamCCD = fits.open('../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../bricks_data/ccds-annotated-90prime-dr9.fits')
print(decamCCD[1].columns)


ColDefs(
    name = 'image_filename'; format = '120A'
    name = 'image_hdu'; format = 'I'
    name = 'camera'; format = '5A'
    name = 'expnum'; format = 'K'
    name = 'plver'; format = '8A'
    name = 'procdate'; format = '19A'
    name = 'plprocid'; format = '7A'
    name = 'ccdname'; format = '3A'
    name = 'object'; format = '35A'
    name = 'propid'; format = '10A'
    name = 'filter'; format = '1A'
    name = 'exptime'; format = 'E'
    name = 'mjd_obs'; format = 'D'
    name = 'airmass'; format = 'E'
    name = 'fwhm'; format = 'E'
    name = 'width'; format = 'I'
    name = 'height'; format = 'I'
    name = 'ra_bore'; format = 'D'
    name = 'dec_bore'; format = 'D'
    name = 'crpix1'; format = 'E'
    name = 'crpix2'; format = 'E'
    name = 'crval1'; format = 'D'
    name = 'crval2'; format = 'D'
    name = 'cd1_1'; format = 'E'
    name = 'cd1_2'; format = 'E'
    name = 'cd2_1'; format = 'E'
    name = 'cd2_2'; format = 'E'
    name = 'yshift'; format = 'L'
    name = 

In [3]:
dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data

In [4]:
ra0 = dataDecam.field('ra0')
dec0 = dataDecam.field('dec0')
ra1 = dataDecam.field('ra1')
dec1 = dataDecam.field('dec1')
ra2 = dataDecam.field('ra2')
dec2 = dataDecam.field('dec2')
ra3 = dataDecam.field('ra3')
dec3 = dataDecam.field('dec3')

In [16]:
import pandas as pd
df = pd.read_csv('../bricks_data/galaxy_catalogue_sample.csv')

ra = df["RA"].to_numpy(copy=True)
dec = df["DEC"].to_numpy(copy=True)

raDec = np.stack((ra,dec), axis=1)

raDec.shape

(379416, 2)

In [20]:
import matplotlib.path as mplPath


for no in range(len(ra0)):

    x0 = ra0[no]
    x1 = ra1[no]
    x2 = ra2[no]
    x3 = ra3[no]

    y0 = dec0[no]
    y1 = dec1[no]
    y2 = dec2[no]
    y3 = dec3[no]

    boundingBoxPath = mplPath.Path(np.array([[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]))

    no_objects_in_ccd = boundingBoxPath.contains_points(raDec).sum()
    if no_objects_in_ccd > 0:
        print(no, no_objects_in_ccd)

791 161
797 92
855 104
856 131
861 45
862 65
976 473
982 314
988 167
995 451
1002 502
1008 641
1013 717
1031 675
1037 409
1043 183
1050 390
1057 698
1063 1054
1261 78
1265 181
1269 585
1275 1014
1281 920
1287 365
1294 442
1301 348
1312 316
1317 442
1319 400
1332 143
1333 20
1338 142
1339 36
1438 13
1498 13
1512 145
1518 193
2267 36
2268 98
2273 171
2454 18
2455 100
2458 204
2497 15
2498 24
2503 93
2504 153
2509 99
2510 1
2515 37
2518 151
7285 22
7286 25
7292 127
7293 89
7299 59
7688 94
7689 30
7693 33
7694 142
7904 47
7905 48
7910 169
7919 89
7943 123
7944 6
7950 215
7951 21
7978 9
8003 122
8004 6
8010 215
8011 21
8038 9
8297 4
8303 225
8310 98
8357 5
8363 224
8370 98
8696 51
8699 84
8756 51
8759 86
8903 53
8910 137
8917 1
8941 8
8984 92
8989 53
8990 140
8994 7
9314 18
9315 48
9320 90
9321 115
9327 47
9455 60
9461 102
9462 93
9467 31
9468 35
9797 1
9803 31
10071 72
10072 18
10075 23
10076 195
10079 10
10812 135
10818 189
10936 832
10942 754
10955 98
10962 197
10973 107
10980 210
10981 

KeyboardInterrupt: 

In [28]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))


In [42]:
import healpy as hp

NSIDE = 256
NPIX = hp.nside2npix(NSIDE)

pixels_for_ccd = []

# In this function, I am first converting every CCD into a given polygon using its corner coordinates
# Afterwards, I am using the ang2vec function and the hp.query_polygon function, to identify every pixel that features the given CCD
# Finally, I am storing these pixels per CCD in a list

for no in range(100000):#(len(ra0)):
    x_coord_polygon = np.array([ra0[no],ra1[no], ra2[no], ra3[no]])
    y_coord_polygon = np.array([dec0[no],dec1[no], dec2[no], dec3[no]])
    theta, phi = raDec2thetaPhi(x_coord_polygon, y_coord_polygon)

    ccd_vertices_3d = hp.ang2vec(theta=theta, phi=phi)
    pixels_for_ccd.append(hp.query_polygon(nside=NSIDE, vertices=ccd_vertices_3d, inclusive=True))
    if no % 5000 == 0:
        print(no)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [70]:
pixel2ccd_dict = {}

# In this function, I am first converting every CCD into a given polygon using its corner coordinates
# Afterwards, I am using the ang2vec function and the hp.query_polygon function, to identify every pixel that features the given CCD
# Then, I am creating a dictionary that lists all CCDs for a given pixel

for no in range(len(ra0)):#range(len(ra0)):
    x_coord_polygon = np.array([ra0[no],ra1[no], ra2[no], ra3[no]])
    y_coord_polygon = np.array([dec0[no],dec1[no], dec2[no], dec3[no]])
    theta, phi = raDec2thetaPhi(x_coord_polygon, y_coord_polygon)

    ccd_vertices_3d = hp.ang2vec(theta=theta, phi=phi)

    # Now, Im getting all pixels that a given CCD covers
    try:
        pixels_for_ccd = hp.query_polygon(nside=NSIDE, vertices=ccd_vertices_3d, inclusive=True)
    except:
        continue
    #print(pixels_for_ccd)
    # Now, im reverting this operation
    # For every pixel that a given CCD cuts,
    for pixel in pixels_for_ccd:
        if pixel not in pixel2ccd_dict:
            pixel2ccd_dict[pixel] = []
        pixel2ccd_dict[pixel].append(no)

    if no % 40000 == 0:
        print(no)


    '''if no % 5000 == 0:
        print(no)
        break'''

0
40000
80000
120000
160000
200000
240000
280000
320000
360000
400000
440000
480000
520000
560000
600000
640000
680000
720000
760000
800000
840000
880000
920000
960000
1000000
1040000
1080000
1120000
1160000
1200000
1240000
1280000
1320000
1360000
1400000
1440000
1480000
1520000
1560000
1600000
1640000
1680000
1720000
1760000
1800000
1840000
1880000
1920000
1960000
2000000
2040000
2080000
2120000
2160000
2200000
2240000
2280000
2320000
2360000
2400000
2440000
2480000
2520000
2560000
2600000
2640000
2680000
2720000
2760000
2800000
2840000
2880000
2920000
2960000
3000000
3040000
3080000
3120000
3160000
3200000
3240000
3280000
3320000
3360000
3400000
3440000
3480000
3520000
3560000
3600000
3640000
3680000
3720000
3760000
3800000
3840000
3880000
3920000
3960000
4000000
4040000
4080000
4120000
4160000
4200000
4240000
4280000
4320000
4360000
4400000
4440000
4480000
4520000
4560000
4600000
4640000
4680000
4720000
4760000
4800000
4840000
4880000
4920000
4960000
5000000
5040000
5080000
5120000


#### Plotting the Boundaries of the different CCDs that are associated to an arbitrary pixel

In [75]:
pixel_number = 620311
print(pixel2ccd_dict.keys())
print(len(pixel2ccd_dict.keys()))

import matplotlib.pyplot as plt
for i, keys in enumerate(pixel2ccd_dict.keys()):
    break
    ccd = pixel2ccd_dict[keys]
    print("Number of CCDs that are cutting pixel number ", keys, ":", len(ccd))
    for cc in ccd:
    #coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
        xs = [ra0[cc],ra1[cc], ra2[cc], ra3[cc],ra0[cc] ]
        ys = [dec0[cc],dec1[cc], dec2[cc], dec3[cc],dec0[cc]]
        plt.plot(xs,ys)

    if i > 3:
        break
plt.show()

dict_keys([655892, 655893, 656912, 656913, 655894, 655895, 655896, 656914, 656915, 656916, 656911, 657927, 657928, 657929, 657930, 657931, 657932, 658939, 658940, 658941, 658942, 658943, 656917, 657933, 658944, 657926, 658938, 659946, 659947, 659948, 659949, 659950, 659951, 658945, 659952, 659953, 660950, 660951, 660952, 660953, 660954, 660955, 660956, 659945, 660949, 661951, 661952, 661953, 661954, 661955, 659954, 660957, 660958, 661949, 661950, 661956, 661957, 662946, 662947, 662948, 662949, 662950, 662951, 662952, 662953, 663938, 663939, 663940, 663941, 663942, 663943, 663944, 664926, 664927, 664928, 664929, 664930, 664931, 664932, 665911, 665912, 665913, 665914, 665915, 660959, 660960, 661958, 661959, 661960, 662954, 662955, 662956, 663945, 663946, 663947, 661961, 662957, 663948, 664933, 664934, 664935, 663949, 664936, 664937, 665916, 665917, 665918, 665919, 665920, 664938, 665921, 665922, 666893, 666894, 666895, 666896, 666897, 666898, 666899, 666900, 666901, 667870, 667871, 66787

In [ ]:
print(NPIX)
print(pixel2ccd_dict)

Now, trying to actually get all ccds per pixel

In [ ]:
print(len(pixels_for_ccd))
print(NPIX)
pixel2ccd_dict = {}
#Iterate through all pixels
for i in range(NPIX):
    ccds_per_pixel = []

    #Iterate through the ccds and which pixels they are associated to
    for j, elem in enumerate(pixels_for_ccd):

        if i in elem:
            ccds_per_pixel.append(j)

    pixel2ccd_dict[i] = ccds_per_pixel

    if i == 200:
        break
print(len(pixel2ccd_dict))

In [49]:
for i in range(201):
    print(pixel2ccd_dict[i])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
expnumD = dataDecam.field('expnum')
expnumM = dataMosaic.field('expnum')
expnumB = dataBass.field('expnum')
expnum = np.concatenate((expnumB,expnumD, expnumM))
print(len(expnumB))